In [12]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from farmsize import data_prep, db_scan, mapping
from shapely.geometry import Point

In [22]:
ipums_level1 = gpd.read_file('./data/mapping/ipums/world_geolev1_2020/world_geolev1_2020.shp')
ipums_level2 = gpd.read_file('./data/mapping/ipums/world_geolev2_2020/world_geolev2_2020.shp')



In [42]:
ipums_level2.loc[ipums_level2["CNTRY_NAME"].str.contains("Sierra"),:]
# ipums_level2[ipums_level2["CNTRY_NAME"]=="Ethiopia"].plot()

,CNTRY_NAME,ADMIN_NAME,CNTRY_CODE,GEOLEVEL2,BPL_CODE,geometry
12076,Sierra Leone,Luawa,694,694011001,15160.0,"POLYGON ((-10.45587 8.36333, -10.45577 8.34013..."
12077,Sierra Leone,"Kpeje West, Njaluahun",694,694011002,15160.0,"POLYGON ((-10.74217 8.22463, -10.74627 8.22063..."
12078,Sierra Leone,"Kissi Kama, Kissi Teng",694,694011003,15160.0,"POLYGON ((-10.31651 8.50065, -10.31611 8.50057..."
12079,Sierra Leone,Jawie,694,694011004,15160.0,"POLYGON ((-10.80517 7.90063, -10.80957 7.89963..."
12080,Sierra Leone,"Kpeje Bongre, Penguia, Yawei",694,694011005,15160.0,"POLYGON ((-10.65076 8.42304, -10.65013 8.42257..."
...,...,...,...,...,...,...
12179,Sierra Leone,Western - urban - East 2,694,694042004,15160.0,"POLYGON ((-13.28836 8.49792, -13.28503 8.49456..."
12180,Sierra Leone,Western - urban - East 3,694,694042005,15160.0,"POLYGON ((-13.28836 8.49792, -13.28503 8.49456..."
12181,Sierra Leone,Western - urban - West 1,694,694042006,15160.0,"POLYGON ((-13.28836 8.49792, -13.28503 8.49456..."
12182,Sierra Leone,Western - urban - West 2,694,694042007,15160.0,"POLYGON ((-13.28836 8.49792, -13.28503 8.49456..."


# Loading and Viewing Data

In [ ]:
indicator_data = pd.read_csv("./data/RHoMIS_Indicators.csv", encoding="latin") 
countries_iso_2= ["ET","KE","TZ", "UG","BF","ML","GH", "SL", "NG", "RW", "BI",]
countries_iso_3 = ["ETH", "KEN", "TZA", "UGA", "BFA", "MLI", "GHA",  "SLE", "NGA","RWA", "BDI"]
ipums_country_names = ["Ethiopia", "Kenya",]
country_name_mapping= {
    "ETH":{"ipums": "Ethiopia", "iso_2":"ET", "iso_3":"KEN"}, 
    "KEN": : {"ipums":"Kenya", "iso_2":"KE", "iso_3":"KEN"}, 
    "TZA": {"ipums":"Tanzania", "iso_2":"TZ", "iso_3":"TZA"}, 
    "UGA": {"ipums":"Uganda", "iso_2":"UG", "iso_3":"UGA"}, 
    "BFA": {"ipums":"Burkina Faso", "iso_2":"BF", "iso_3":"BFA"}, 
    "MLI": {"ipums":"Mali", "iso_2":"ML", "iso_3":"MLI"}, 
    "GHA": {"ipums":"Ghana", "iso_2":"GH", "iso_3":""},
    "SLE": {"ipums":"Sierra Leone", "iso_2":"SL", "iso_3":"SLE"}, 
    "NGA": {"ipums":"", "iso_2":"", "iso_3":""},
    "RWA": {"ipums":"", "iso_2":"", "iso_3":""}, 
    "BDI": {"ipums":"", "iso_2":"", "iso_3":""}}
indicator_data = data_prep.subset_data(indicator_data, complete_gps=True, countries=countries_iso_2)

In [ ]:
# World Shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
# Generated using the function adminstrative_boundaries = mapping.get_admin_boundaries_multiple_countries(...)
admin_1 = mapping.read_admin_boundaries_file('./data/mapping/subnational_geometries.csv')
admin_1

In [ ]:
# Spatially joining the two datasets
indicator_data["geometry"] = [Point(xy) for xy in zip(indicator_data["GPS_LON"], indicator_data["GPS_LAT"])]
geo_indicator = gpd.GeoDataFrame(indicator_data)
geo_indicator = geo_indicator.sjoin(world, how="left", op="within")


In [ ]:
world

In [ ]:
# Plot the points on a map
#same as
#fig= plt.figure()
#ax=fig.add_subplot()
fig, ax =  plt.subplots(figsize=(15,15))
ax.set_aspect('equal')
#Plot map layer
world.loc[world["continent"]=="Africa",].plot(ax=ax,color="white", edgecolor="black")

world.loc[world["iso_a3"].isin(countries_iso_3),].plot(ax=ax,color="blue", edgecolor="black", alpha=0.1)

#Plot Points
geo_indicator.plot(ax=ax, marker=0, color='black', markersize=5)

fig.show()
fig.savefig('./outputs/exploratory/map.png')


In [ ]:
# Grouping by 
row_subsets = indicator_data["LandCultivated"].notna() & indicator_data["LandCultivated"].between(0.05,100)
column_subsets= ["ID_COUNTRY", "LandCultivated"]
grouping="ID_COUNTRY"

fig, ax = plt.subplots(figsize=(10,10))
plt.tight_layout()
ax.set_title("KDE of Land ")
ax.set_xlabel("Land Cultivated (ha)")
ax.set_ylabel("Density")
ax.set_xlim([0,25])
ax.set_ylim([0,0.6])
indicator_data.loc[row_subsets,:].groupby("ID_COUNTRY")["LandCultivated"].plot(kind="kde", ax=ax, legend=True)
fig.savefig("./outputs/exploratory/land_size_kde_all_countries.png", bbox_inches="tight")

# Clustering Households Spatially

In [ ]:
cluster_labels = db_scan.cluster_gps_points(indicator_data, "GPS_LON", "GPS_LAT", epsilon=0.1)


In [ ]:
mapping.get_admin_boundaries_multiple_countries(["ETH","KEN"])




In [ ]:
import requests

def get_country_admin_boundaries(iso_a_3):
    # See documentation on GeoBoundaries API
    url = "https://www.geoboundaries.org/gbRequest.html?ISO="+iso_a_3+"ADM=ADM1"
    r = requests.get(url)
    downloand_url  = r.json()[0]['gjDownloadURL']
    geoBoundary = requests.get(downloand_url).json()

    names = [feature["properties"]["shapeISO"] for feature in geoBoundary["features"] ]
    shapeISO = [feature["properties"]["shapeName"] for feature in geoBoundary["features"] ]
    shapeID = [feature["properties"]["shapeID"] for feature in geoBoundary["features"] ]
    shapeType = [feature["properties"]["shapeType"] for feature in geoBoundary["features"] ]
    geometry = [shape(feature["geometry"]) for feature in geoBoundary["features"] ]

    geo_data_frame = gpd.GeoDataFrame(data={
        "iso_a3": iso_a_3,
        "region_name": names,
        "shapeISO": shapeISO,
        "shapeID": shapeID,
        "shapeType": shapeType,
        "geometry": geometry
    })

    return geo_data_frame





In [ ]:
world.loc[world["continent"]=="Africa","iso_a3"].values









In [ ]:
geo_data_frame